# 1. Import Dependencies

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# 3. Encode and Calculate Sentiment

In [3]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [5]:
tokens[0]

tensor([  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
        11838,   102])

In [7]:
tokenizer.decode(tokens[0])

'[CLS] it was good but couldve been better. great [SEP]'

In [8]:
result = model(tokens)

In [9]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [10]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [15]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [21]:
reviews

['Cooking food is like being a person, you must use real ingredients and select the best. Only careful cooking can make a delicious dish.',
 "Amazing ambience. Coffee was the best we've had in Sydney. The service and food also great. As must stop for breakfast.",
 "Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking an

# 5. Load Reviews into DataFrame and Score

In [22]:
import numpy as np
import pandas as pd

In [23]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [31]:
df.iloc[4]

review       We came for brunch and they ran out of seven s...
sentiment                                                    2
Name: 4, dtype: object

In [26]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [27]:
sentiment_score(df['review'].iloc[1])

5

In [28]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [29]:
df

,review,sentiment
0,"Cooking food is like being a person, you must ...",4
1,Amazing ambience. Coffee was the best we've ha...,5
2,Very cute coffee shop and restaurant. They hav...,4
3,Six of us met here for breakfast before our wa...,4
4,We came for brunch and they ran out of seven s...,2
5,Ricotta hot cakes! These were so yummy. I ate ...,5
6,Great food amazing coffee and tea. Short walk ...,5
7,We came for brunch twice in our week-long visi...,4
8,Great place with delicious food and friendly s...,5
9,The food was delicious. The ricotta pancakes w...,4


In [32]:
df["review"].iloc[4]

"We came for brunch and they ran out of seven separate menu items. We tried ordering multiple times and each time were let down with them not having what we wanted. It's understandable to not have certain things, but to have half of your menu not available is a bit ridiculous. When we finally were able to get something it was good, but we had to wait around 10-15 minutes after ordering to be told that they were out of something. Disappointing because of high expectations."